In [1]:
# ───────────────────────────────
# 📦 Standard Libraries
# ───────────────────────────────
import os
import sys
import math
import time
import csv
import pickle
from datetime import datetime, timedelta
from collections import Counter

# ───────────────────────────────
# 📚 Data Handling & Utilities
# ───────────────────────────────
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
import h5py
from joblib import dump, load

# ───────────────────────────────
# 📊 Visualization
# ───────────────────────────────
import matplotlib.pyplot as plt
import seaborn as sns

# ───────────────────────────────
# 📈 Machine Learning
# ───────────────────────────────
from sklearn.model_selection import (
    train_test_split, RandomizedSearchCV, GridSearchCV, cross_val_score
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, roc_curve, roc_auc_score, auc,
    classification_report, confusion_matrix
)
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.under_sampling import RandomUnderSampler

# ───────────────────────────────
# 🌍 Seismology & Signal Processing
# ───────────────────────────────
import obspy
from obspy import UTCDateTime
from obspy.geodetics.base import gps2dist_azimuth
from obspy.clients.fdsn import Client
from obspy.signal.filter import envelope
from scipy import stats, signal

# ───────────────────────────────
# 🔗 External Tools
# ───────────────────────────────
from zenodo_get import zenodo_get

# ───────────────────────────────
# 🤖 Deep Learning
# ───────────────────────────────
import torch
import torch.nn.functional as F


# ───────────────────────────────
# 🛠 Custom Utilities
# ───────────────────────────────
sys.path.insert(0, '/home/ak287/PNW_Seismic_Event_Classification/deep_learning/scripts')
from neural_network_architectures import SeismicCNN_2d

from utils import (
    extract_waveforms,
    compute_spectrogram,
    normalize_spectrogram_minmax,
    return_train_val_loaders,
    plot_confusion_matrix_and_cr,
    train_model,
    WaveformPreprocessor
)

# Pandas display options
pd.set_option('display.max_columns', None)


sys.path.append('/home/ak287/seisbench/seisbench/models')
import seisbench.models as sbm



# for extracting unique stations
import re
from pathlib import Path

cuda


In [3]:
cat_exp = pd.read_csv('../../src/pnw_explosion_2023_2025.csv')

In [4]:
cat_exp

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,id,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2025-06-18T17:03:42.500Z,43.261833,-123.554333,-0.47,0.65,md,6,92.0,0.14880,0.20,uw,uw62122302,2025-06-18T19:25:25.850Z,"7 km WNW of Melrose, Oregon",explosion,0.65,31.61,0.198924,5,reviewed,uw,uw
1,2025-06-17T19:30:03.810Z,47.113667,-123.102000,-0.23,1.36,ml,13,112.0,0.06747,0.26,uw,uw62129981,2025-06-18T22:28:02.320Z,"11 km S of Shelton, Washington",explosion,0.51,31.61,0.149711,7,reviewed,uw,uw
2,2025-06-17T17:26:00.490Z,49.457167,-120.531000,-0.51,1.96,ml,9,248.0,0.60690,0.41,uw,uw62129891,2025-06-17T18:48:17.230Z,"1 km W of Princeton, Canada",explosion,1.76,31.61,0.089090,8,reviewed,uw,uw
3,2025-06-17T17:20:48.310Z,44.581167,-116.635167,-1.50,1.32,mh,3,141.0,0.54990,0.09,uw,uw62129886,2025-06-17T18:21:47.580Z,"3 km ENE of Cambridge, Idaho",explosion,0.71,31.61,NaN,0,reviewed,uw,uw
4,2025-06-16T22:10:25.770Z,46.888667,-123.767000,-0.23,1.13,ml,11,151.0,0.08369,0.07,uw,uw62129511,2025-06-16T23:54:26.460Z,"7 km S of Cosmopolis, Washington",explosion,0.43,31.61,0.187089,9,reviewed,uw,uw
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1658,2023-01-04T22:38:55.520Z,47.892000,-122.710000,-0.03,1.66,ml,18,50.0,0.09584,0.30,uw,uw61901426,2024-07-15T19:12:02.530Z,"4 km SSW of Port Ludlow, Washington",explosion,0.51,31.61,0.113042,8,reviewed,uw,uw
1659,2023-01-04T00:00:04.550Z,48.509667,-123.499667,-0.24,0.92,ml,4,156.0,0.14380,0.26,uw,uw61901346,2023-01-05T20:25:30.460Z,"6 km NNW of Six Mile, Canada",explosion,1.30,31.61,0.088836,4,reviewed,uw,uw
1660,2023-01-03T21:27:02.710Z,43.604667,-123.063667,-0.53,1.37,ml,8,149.0,0.08158,0.06,uw,uw61901331,2023-01-03T22:10:43.930Z,"17 km E of Yoncalla, Oregon",explosion,0.50,31.61,0.065416,7,reviewed,uw,uw
1661,2023-01-03T20:41:01.760Z,47.006500,-122.201833,-0.54,1.47,ml,16,246.0,0.02069,0.20,uw,uw61901326,2023-01-03T22:05:37.620Z,"2 km NE of Kapowsin, Washington",explosion,0.98,31.61,0.057117,9,reviewed,uw,uw


## Loading the trained model

In [6]:
## setting up some important parameters (not to be changed)
num_channels = 3
dropout = 0.9
# Check if a GPU is available
device = "cuda"



model_quakexnet_2d = SeismicCNN_2d(num_classes=4, num_channels=num_channels,dropout_rate=dropout).to(device)  # Use 'cuda' if you have a GPU available

# Load the saved model state dict (weights)
saved_model_quakexnet_2d = torch.load('../trained_models/best_model_new_augmented_esec_SeismicCNN_2d.pth', map_location=torch.device('cpu'))  # No 'weights_only' argument
model_quakexnet_2d.load_state_dict(saved_model_quakexnet_2d)
model_quakexnet_2d.to(device)
model_quakexnet_2d.eval()


model = model_quakexnet_2d
model.to('cuda')

SeismicCNN_2d(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=15360, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=4, bias=True)
  (softmax): Softmax(dim=1)
  (dropout): Dropout(p=0.9, inplace=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1_bn): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2_bn): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)